Objective:
Answer any question based on the content of a PDF. The approach uses  embeddings persisted in Pinecone and Cosine similarity.
This code is originally at: https://github.com/jose-leon-cognigy/langchain-tutorials

In [ ]:
!pip install langchain
!pip install unstructured
!pip install pdfminer.six
!pip install torchvision

# Install Unstructured is a headache, follow the guide here: 
# https://github.com/Unstructured-IO/unstructured#coffee-getting-started  (you need C++ Build tools installed)
!pip install "unstructured[local-inference]   



In [7]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

### Load your data

In [8]:
loader = UnstructuredPDFLoader("../data/field-guide-to-data-science.pdf")
# loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")

In [9]:
data = loader.load()

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


In [10]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

You have 1 document(s) in your data
There are 201014 characters in your document


### Chunk your data up into smaller documents

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [12]:
print (f'Now you have {len(texts)} documents')

Now you have 240 documents


### Create embeddings of your documents to get ready for semantic search

In [3]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

C:\Users\JoseArturoValdiviaLe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [1]:
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv("PINECONE_API_ENV")

In [4]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [5]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchain"

In [13]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [14]:
query = "What are examples of good data science teams?"
docs = docsearch.similarity_search(query, include_metadata=True)

### Query those docs to get your answer back

In [15]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [16]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [19]:
query = "What is the collect stage of data maturity?"
# query = "What are examples of good data science teams?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [20]:
chain.run(input_documents=docs, question=query)

' The collect stage of data maturity focuses on collecting internal or external datasets. An example of this stage is gathering sales records and corresponding weather data.'